In [1]:
import numpy as np
np.float_ = np.float64
from docplex.mp.model import Model

In [2]:
mdl = Model('BEP')

In [3]:
fileName = "instancias/InstanceBEP-2-32-5-18_7.txt"
f = open(fileName, 'r')

firstLine = f.readline().strip("\n").split(" ")
b = int(firstLine[0].split(":")[0])
c = int(firstLine[1])

secondLine = f.readline().strip("\n").split(":")
y = int(secondLine[0])
Y = list(map(int, secondLine[1].strip().split(" ")))

thirdLine = f.readline().strip("\n").split(":")
p = int(thirdLine[0])
personasTotal = int(thirdLine[1])
P = list(map(int, thirdLine[2].strip().split(" ")))

fourthLine = f.readline().strip("\n").split(":")
s = int(fourthLine[0])
sumaCapacidadesRefugios = int(fourthLine[1])
S = list(map(int, fourthLine[2].strip().split(" ")))

y_range = range(y)
p_range = range(y, y + p)
s_range = range(y + p, y + p + s)

f.readline() # Empty line
distanciasEstacionesPuntosEncuentros = []

for _ in range(y):
    distanciasEstacionPuntosEncuentros = f.readline().strip("\n").split(":")
    distanciasEstacionesPuntosEncuentros.append(distanciasEstacionPuntosEncuentros[1].strip().split(" "))

distanciasEstacionesPuntosEncuentros = np.array(distanciasEstacionesPuntosEncuentros).astype(int)

f.readline() # Empty line
distanciasPuntosEncuentrosRefugios = []

for _ in range(p):
    distanciasPuntoEncuentroRefugios = f.readline().strip("\n").split(":")
    distanciasPuntosEncuentrosRefugios.append(distanciasPuntoEncuentroRefugios[1].strip().split(" "))

distanciasPuntosEncuentrosRefugios = np.array(distanciasPuntosEncuentrosRefugios).astype(int)

In [4]:
N = [i for i in range(y + p + s)] # Nodos
A = [(i, j) for i in range(y) for j in p_range if i != j] # Arcos partida -> punto de encuentro
A += [(i, j) for i in p_range for j in s_range if i != j] # Arcos punto de encuentro -> refugio
A += [(j, i) for i in p_range for j in s_range if i != j] # Arcos refugio -> punto de encuentro

In [5]:
T = mdl.integer_var(name='T')
mdl.minimize(T)

t_evac = 10

In [6]:
tau = {}
for i, j in A:
    if i in y_range:
        tau[(i, j)] = int(distanciasEstacionesPuntosEncuentros[i][j - y])
    else:
        if (j, i) in tau:
            tau[(i, j)] = tau[(j, i)]
        else:
            tau[(i, j)] = tau[(j, i)] = int(distanciasPuntosEncuentrosRefugios[i - y][j - p - y]) # Tiempo de viaje

In [7]:
# x_{ij}^{vt}: x es 1 si en el viaje t el bus m va de i a j, 0 en caso contrario
x = mdl.binary_var_dict(((i, j, m, t) for i, j in A for m in range(b) for t in range(t_evac)), name="x")

In [8]:
# b_{j}^{vt}: numero de evacuados desde el nodo j que serán transportados por el bus m (o si j es un refugio, liberados) después del viaje t
b_ = mdl.integer_var_dict(((j, m, t) for j in N for m in range(b) for t in range(t_evac)), name='b')

In [9]:
for t1 in range(1, t_evac, 2): # No pueden haber viajes desde un refugio a un punto de encuentro en un viaje impar.
    for m in range(b):
        for i, j in A:
            if i in s_range:
                mdl.add_constraint(x[i, j, m, t1] == 0)

for t1 in range(2, t_evac, 2): # No pueden haber viajes desde un punto de encuentro a un refugio en un viaje par.
    for m in range(b):
        for i, j in A:
            if i in p_range:
                mdl.add_constraint(x[i, j, m, t1] == 0)

for m in range(b): # 2) El tiempo de evacuación resultante debe ser mayor o igual al costo máximo de cualquier viaje.
    sumaTotal = 0
    for t1 in range(t_evac):
        for i, j in A:
            sumaTotal += tau[i, j] * x[i, j, m, t1]
    mdl.add_constraint(sumaTotal <= T)

for m in range(b): # 3) se asegura que un bus que viaja al nodo j en el viaje t deja el nodo j en el viaje t + 1. Desde un punto de partida a un punto de encuentro a un refugio
    for t1 in range(1):
        for j in p_range:
            otra = 0
            suma = 0
            for i in y_range:
                otra += x[i, j, m, t1]
            for k in s_range:
                suma += x[j, k, m, t1 + 1]
            mdl.add_constraint(otra == suma)

for m in range(b): # 3) se asegura que un bus que viaja al nodo j en el viaje t deja el nodo j en el viaje t + 1. Desde un refugio a un punto de encuentro a un refugio
    for t1 in range(2, t_evac - 1, 2):
        for j in p_range:
            otra = 0
            suma = 0
            for i in s_range:
                otra += x[i, j, m, t1]
            for k in s_range:
                suma += x[j, k, m, t1 + 1]
            mdl.add_constraint(otra == suma)

for m in range(b): # 4) no se requiere que el bus deje el refugio s
    for t1 in range(1, t_evac - 1, 2):
        for j in s_range:
            otra = 0
            suma = 0
            for i in p_range:
                otra += x[i, j, m, t1]
            for k in p_range:
                suma += x[j, k, m, t1 + 1]
            mdl.add_constraint(otra >= suma)

for m in range(b): # 5) Un bus solamente puede hacer un viaje a la vez.
    for t1 in range(t_evac - 1):
        suma = 0
        for i, j in A:
            suma += x[i, j, m, t1]
        mdl.add_constraint(suma <= 1)
        
for i in y_range: # 6) Cada bus debe partir desde la estacion en su primer viaje.
    for m in range(Y[i]):
        suma = 0
        for j in p_range:
            offset = Y[i - 1] if i != 0 else 0
            suma += x[i, j, m + offset, 0]
        mdl.add_constraint(suma == 1)

for m in range(b): # 7) Si un bus no sale de su estacion al comienzo de la evacuacion, no sale tampoco en viajes posteriores.
    for t1 in range(1, t_evac):
        for y1, j in A:
            if y1 in y_range:
                mdl.add_constraint(x[y1, j, m, t1] == 0)

for m in range(b): # 8) No se permite que el ultimo viaje de un bus termine en un punto de encuentro.
    for i, p1 in A:
        if p1 in p_range:
            mdl.add_constraint(x[i, p1, m, t_evac - 1] == 0)

for t1 in range(t_evac): # 9) Un bus solo puede recoger a los evacuados desde el nodo j si ha viajado al nodo j.
    for m in range(b):
        for j in N:
            suma = 0
            for i in N: 
                if (i, j) in A:
                    suma += x[i, j, m, t1]
                    #print(x[i, j, m, t1].x*c, end=", ")
            if (suma):
                #print(j, m, t1, suma)
                mdl.add_constraint(b_[j, m, t1] == c * suma)

for t2 in range(t_evac): # 10) Capacidad del bus
    for m in range(b):
        suma1 = 0
        for j in p_range:
            for l in range(t2 + 1):
                suma1 += b_[j, m, l]
        suma2 = 0
        for k in s_range:
            for l in range(t2 + 1):
                suma2 += b_[k, m, l]
        mdl.add_constraint(0 <= suma1 - suma2)
        mdl.add_constraint(suma1 - suma2 <= c)

for j in range(s): # 11) Capacidad del refugio.
    suma = 0
    for m in range(b):
        for t1 in range(t_evac):
            suma += b_[j + y + p, m, t1]
    mdl.add_constraint(suma <= S[j])

for j in range(p): # 12) Todos los evacuados deben ser recogidos de los puntos de encuentro.
    suma = 0
    for m in range(b):
        for t1 in range(t_evac):
            suma += b_[j + y, m, t1]
    mdl.add_constraint(suma == P[j])

for m in range(b): # 13) Todos los evacuados deben ser llevados a los refugios.
    suma1 = 0
    for p1 in p_range:
        for t1 in range(t_evac):
            suma1 += b_[p1, m, t1]
    suma2 = 0
    for s1 in s_range:
        for t2 in range(t_evac):
            suma2 += b_[s1, m, t2]
    mdl.add_constraint(suma1 == suma2)

In [10]:
solution = mdl.solve(log_output=True)

-- cannot find parameters matching version: , using: 22.1.1.0


DOcplexException: Cannot solve model: no CPLEX runtime found.

In [ ]:
#print(solution.get_objective_value())
if solution:
    #print(solution)
    for m in range(b):
        print(f"Bus {m + 1}:", end=" ")
        tiempo = 0
        for t1 in range(t_evac):
            for i, j in A:
                if x[i, j, m, t1].solution_value == 1:
                    if t1 == 0:
                        print(f"{i + 1} -> {j + 1 - y}", end=" | ")
                    else:
                        if t1 % 2 == 0:
                            print(f"{i + 1 - y} -> {j + 1 - y - p}", end=" | ")
                    tiempo += tau[i, j]
        print(f"Time: {tiempo}")
